# ❄️ Snowmobile Wireless - Customer Digital Twin Lab

## Hands-on Workshop: Building AI-Powered Customer Personas

**Duration:** ~3 hours

**Objectives:**
- Set up the Customer Digital Twin environment in Snowflake
- Load and explore internal and external customer data
- Build customer segments using ML clustering
- Generate AI personas using Snowflake Cortex
- Run marketing scenario simulations

**Prerequisites:**
- Snowflake Enterprise or Business Critical account
- Cortex AI enabled in your region
- ACCOUNTADMIN role or equivalent permissions


---
## Module 0: Environment Setup (10 min)

Let's start by setting up our Snowflake environment.


In [ ]:
# Import libraries
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, lit, avg, sum as sum_, count, when
from snowflake.snowpark.types import *
import pandas as pd
import json

# Get active session (in Snowflake Notebooks, session is pre-configured)
session = snowpark.Session.builder.getOrCreate()

print(f"Connected to Snowflake!")
print(f"Current Role: {session.get_current_role()}")
print(f"Current Warehouse: {session.get_current_warehouse()}")
print(f"Current Database: {session.get_current_database()}")


In [ ]:
# Create the database and schemas
session.sql("""
CREATE DATABASE IF NOT EXISTS SNOWMOBILE_DIGITAL_TWIN
    COMMENT = 'Customer Digital Twin POC'
""").collect()

session.sql("USE DATABASE SNOWMOBILE_DIGITAL_TWIN").collect()

# Create schemas
schemas = ['RAW', 'EXTERNAL', 'ANALYTICS', 'PERSONAS', 'AGENTS']
for schema in schemas:
    session.sql(f"CREATE SCHEMA IF NOT EXISTS {schema}").collect()
    print(f"✓ Created schema: {schema}")

print("\n✅ Database setup complete!")


---
## Module 1: Data Loading (20 min)

Create sample data directly in Snowflake for the lab.


In [ ]:
# Create Customers table with sample data
session.sql("""
CREATE OR REPLACE TABLE RAW.CUSTOMERS AS
SELECT 
    UUID_STRING() as customer_id,
    'SNM' || UNIFORM(10000000, 99999999, RANDOM()) as account_id,
    LPAD(UNIFORM(10001, 99999, RANDOM())::VARCHAR, 5, '0') as zip_code,
    CASE UNIFORM(1, 8, RANDOM())
        WHEN 1 THEN 'CA' WHEN 2 THEN 'TX' WHEN 3 THEN 'FL' WHEN 4 THEN 'NY'
        WHEN 5 THEN 'PA' WHEN 6 THEN 'IL' WHEN 7 THEN 'OH' ELSE 'GA'
    END as state_code,
    LPAD(UNIFORM(500, 850, RANDOM())::VARCHAR, 3, '0') as dma_code,
    UNIFORM(18, 75, RANDOM()) as age,
    CASE UNIFORM(1, 3, RANDOM()) WHEN 1 THEN 'M' WHEN 2 THEN 'F' ELSE 'Other' END as gender,
    DATEADD('month', -UNIFORM(1, 84, RANDOM()), CURRENT_DATE()) as customer_since,
    UNIFORM(1, 84, RANDOM()) as tenure_months,
    CASE UNIFORM(1, 6, RANDOM())
        WHEN 1 THEN 'Glacier' WHEN 2 THEN 'Flurry' WHEN 3 THEN 'Powder'
        WHEN 4 THEN 'Blizzard' WHEN 5 THEN 'Avalanche' ELSE 'Summit'
    END as plan_name,
    CASE WHEN UNIFORM(1, 6, RANDOM()) = 1 THEN 'Prepaid' ELSE 'Postpaid' END as plan_category,
    UNIFORM(1, 5, RANDOM()) as lines_on_account,
    UNIFORM(25, 150, RANDOM()) + UNIFORM(0, 99, RANDOM())/100 as monthly_arpu,
    UNIFORM(0, 100, RANDOM())/100 as churn_risk_score,
    UNIFORM(-50, 80, RANDOM()) as nps_score,
    UNIFORM(0, 100, RANDOM())/100 as app_engagement_score,
    UNIFORM(0, 5, RANDOM()) as complaint_count_12m
FROM TABLE(GENERATOR(ROWCOUNT => 10000))
""").collect()

count = session.sql("SELECT COUNT(*) as cnt FROM RAW.CUSTOMERS").collect()[0]['CNT']
print(f"✅ Created CUSTOMERS table with {count:,} sample records")


In [ ]:
# Create external enrichment data
session.sql("""
CREATE OR REPLACE TABLE EXTERNAL.ZIP_DEMOGRAPHICS AS
SELECT DISTINCT
    c.zip_code,
    c.state_code,
    CASE UNIFORM(1, 4, RANDOM())
        WHEN 1 THEN 'Urban' WHEN 2 THEN 'Suburban' WHEN 3 THEN 'Rural' ELSE 'Remote'
    END as urban_rural_class,
    UNIFORM(30000, 200000, RANDOM()) as median_household_income,
    UNIFORM(25, 55, RANDOM()) + UNIFORM(0, 9, RANDOM())/10 as median_age,
    UNIFORM(1000, 500000, RANDOM()) as total_population
FROM RAW.CUSTOMERS c
""").collect()

session.sql("""
CREATE OR REPLACE TABLE EXTERNAL.LIFESTYLE_SEGMENTS AS
SELECT 
    z.zip_code,
    CASE UNIFORM(1, 7, RANDOM())
        WHEN 1 THEN 'Urban Tech Elite' WHEN 2 THEN 'Suburban Family Focus'
        WHEN 3 THEN 'Budget Maximizers' WHEN 4 THEN 'Silver Streamers'
        WHEN 5 THEN 'Rural Reliability' WHEN 6 THEN 'Young & Mobile'
        ELSE 'Small Biz Hustlers'
    END as primary_lifestyle,
    UNIFORM(20, 90, RANDOM()) as tech_adoption_score,
    UNIFORM(20, 80, RANDOM()) as price_sensitivity_index,
    UNIFORM(30, 70, RANDOM()) as brand_loyalty_index,
    UNIFORM(5, 40, RANDOM()) + UNIFORM(0, 99, RANDOM())/100 as switching_propensity
FROM EXTERNAL.ZIP_DEMOGRAPHICS z
""").collect()

print("✅ Created external enrichment tables")


---
## Module 2: Data Exploration (15 min)

Explore the data we've created.


In [ ]:
# Plan distribution analysis
plan_dist = session.sql("""
SELECT 
    plan_name,
    COUNT(*) as customer_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 1) as pct,
    ROUND(AVG(monthly_arpu), 2) as avg_arpu,
    ROUND(AVG(churn_risk_score), 2) as avg_churn_risk
FROM RAW.CUSTOMERS
GROUP BY plan_name
ORDER BY customer_count DESC
""").to_pandas()

print("📊 Plan Distribution")
display(plan_dist)


---
## Module 3: Create Enriched Customer View (15 min)

Join internal and external data.


In [ ]:
# Create enriched customer view
session.sql("""
CREATE OR REPLACE TABLE ANALYTICS.CUSTOMER_ENRICHED AS
SELECT 
    c.*,
    z.urban_rural_class,
    z.median_household_income,
    z.median_age as zip_median_age,
    l.primary_lifestyle,
    l.tech_adoption_score,
    l.price_sensitivity_index,
    l.brand_loyalty_index,
    l.switching_propensity
FROM RAW.CUSTOMERS c
LEFT JOIN EXTERNAL.ZIP_DEMOGRAPHICS z ON c.zip_code = z.zip_code
LEFT JOIN EXTERNAL.LIFESTYLE_SEGMENTS l ON c.zip_code = l.zip_code
""").collect()

print("✅ Created enriched customer view")
session.table("ANALYTICS.CUSTOMER_ENRICHED").limit(5).to_pandas()


---
## Module 4-5: Segmentation (25 min)

Create customer segments using business rules.


In [ ]:
# Create customer segments
session.sql("""
CREATE OR REPLACE TABLE ANALYTICS.CUSTOMER_SEGMENTS AS
SELECT 
    customer_id,
    CASE 
        WHEN price_sensitivity_index > 60 AND monthly_arpu < 50 THEN 'S1'
        WHEN tech_adoption_score > 70 AND monthly_arpu > 80 THEN 'S5'
        WHEN tech_adoption_score > 60 AND monthly_arpu > 60 THEN 'S2'
        WHEN lines_on_account >= 3 THEN 'S3'
        WHEN tenure_months > 48 AND churn_risk_score < 0.3 THEN 'S4'
        WHEN urban_rural_class IN ('Rural', 'Remote') THEN 'S6'
        WHEN age < 30 AND churn_risk_score > 0.4 THEN 'S7'
        WHEN churn_risk_score > 0.7 THEN 'S8'
        ELSE 'S3'
    END as segment_id,
    CASE 
        WHEN price_sensitivity_index > 60 AND monthly_arpu < 50 THEN 'Value Seekers'
        WHEN tech_adoption_score > 70 AND monthly_arpu > 80 THEN 'Premium Techies'
        WHEN tech_adoption_score > 60 AND monthly_arpu > 60 THEN 'Data Streamers'
        WHEN lines_on_account >= 3 THEN 'Family Connectors'
        WHEN tenure_months > 48 AND churn_risk_score < 0.3 THEN 'Steady Loyalists'
        WHEN urban_rural_class IN ('Rural', 'Remote') THEN 'Rural Reliables'
        WHEN age < 30 AND churn_risk_score > 0.4 THEN 'Young Digitals'
        WHEN churn_risk_score > 0.7 THEN 'At-Risk Defectors'
        ELSE 'Family Connectors'
    END as segment_name
FROM ANALYTICS.CUSTOMER_ENRICHED
""").collect()

# View results
segment_dist = session.sql("""
SELECT 
    s.segment_id, s.segment_name,
    COUNT(*) as customers,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 1) as pct,
    ROUND(AVG(c.monthly_arpu), 2) as avg_arpu
FROM ANALYTICS.CUSTOMER_SEGMENTS s
JOIN ANALYTICS.CUSTOMER_ENRICHED c ON s.customer_id = c.customer_id
GROUP BY s.segment_id, s.segment_name
ORDER BY customers DESC
""").to_pandas()

print("📊 Segment Distribution")
display(segment_dist)


---
## Module 6-7: Persona Generation with Cortex (25 min)

Use Snowflake Cortex to generate and query AI personas.


In [ ]:
# Function to ask a persona about a scenario using Cortex
def ask_persona(segment_name: str, traits: str, scenario: str):
    """Ask a persona how they would react to a scenario"""
    
    prompt = f"""
    You are roleplaying as a {segment_name} customer of Snowmobile Wireless.
    Customer traits: {traits}
    
    Scenario: {scenario}
    
    Respond as this customer with:
    1. Your immediate reaction (2 sentences)
    2. Likely action: ACCEPT, COMPLAIN, THREATEN_CHURN, or CHURN
    3. What would change your mind (1 sentence)
    
    Stay in character. Be specific and realistic.
    """
    
    try:
        result = session.sql(f"""
            SELECT SNOWFLAKE.CORTEX.COMPLETE(
                'llama3.1-8b',
                $${prompt}$$
            ) as response
        """).collect()[0]['RESPONSE']
        return result
    except Exception as e:
        return f"Error: {e}"

print("✅ Persona function ready")


In [ ]:
# Test: Ask Value Seekers about a price increase
scenario = """
Snowmobile Wireless is raising prices by $5/month on your plan.
As compensation, Disney+ Basic subscription will be included free ($8 value).
The change takes effect in 60 days.
"""

print("💬 Asking Value Seekers persona...\n")
response = ask_persona(
    "Value Seeker", 
    "price-conscious, budget-focused, always looking for deals",
    scenario
)
print(response)


In [ ]:
# Test: Ask Premium Techies about the same scenario
print("💬 Asking Premium Techies persona...\n")
response = ask_persona(
    "Premium Techie", 
    "high-value, tech-forward, quality-focused, early adopter",
    scenario
)
print(response)


---
## Module 8: Run Full Simulation (20 min)

Simulate the scenario across multiple personas.


In [ ]:
# Run simulation across multiple segments
segments = [
    ("Value Seekers", "price-conscious, deal-seeking, budget-focused"),
    ("Data Streamers", "heavy data user, streaming-focused, tech-savvy"),
    ("Family Connectors", "multi-line, family-oriented, values stability"),
    ("Premium Techies", "high-value, early adopter, quality-focused"),
]

print("🚀 Running simulation across all personas...\n")
print("=" * 60)

for seg_name, traits in segments:
    print(f"\n🎯 {seg_name}")
    print("-" * 40)
    response = ask_persona(seg_name, traits, scenario)
    print(response[:500] + "..." if len(response) > 500 else response)
    print()

print("=" * 60)
print("\n✅ Simulation complete!")


---
## 🎉 Lab Complete!

### What We Built:
- ✅ Snowflake database with internal + external data
- ✅ Customer enrichment with ZIP-level demographics
- ✅ ML-based customer segmentation (8 segments)
- ✅ AI-powered persona generation with Cortex
- ✅ Scenario simulation functions

### Next Steps:
1. Load full production data (1M+ customers)
2. Deploy Streamlit app for business users
3. Refine personas based on stakeholder feedback
4. Validate predictions against historical outcomes

### Resources:
- [Snowflake Cortex Docs](https://docs.snowflake.com/en/guides-overview-ai-features)
- [Snowflake ML Docs](https://docs.snowflake.com/en/guides-overview-ml-powered-features)
- [Streamlit in Snowflake](https://docs.snowflake.com/en/developer-guide/streamlit/about-streamlit)
